In [145]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.applications import MobileNet,VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from sklearn.model_selection import train_test_split

In [146]:
train_data_dir = "./ANIMAL_4/train/"
test_data_dir = "./ANIMAL_4/test/"
np_epochs = 8
lr = 1e-2
dc = lr/np_epochs
mom = 0.65
batch = 32

In [147]:
base_model=MobileNet(weights='imagenet',include_top=False)

x=base_model.output
x=GlobalMaxPooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
preds=Dense(4,activation='softmax')(x)

In [148]:
model=Model(inputs=base_model.input,outputs=preds)

In [149]:
for layer in model.layers[:80]:
    layer.trainable=False
for layer in model.layers[80:]:
    layer.trainable=True

In [150]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   horizontal_flip=True,
                                   preprocessing_function= preprocess_input,
                                   validation_split=0.2)

In [151]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = batch,
    class_mode = 'categorical',
    subset='training',
    seed = 42)

Found 800 images belonging to 4 classes.


In [152]:
valid_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = batch,
    class_mode = 'categorical',
    subset='training',
    seed = 42)

Found 800 images belonging to 4 classes.


In [153]:
opt = SGD(learning_rate = lr,decay=dc,momentum=mom)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [154]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch,
    validation_data = valid_generator, 
    validation_steps = valid_generator.samples // batch,
    epochs = np_epochs)

Epoch 1/8
25/25 [==============================] - 52s 2s/step - loss: 7.7748 - accuracy: 0.2734 - val_loss: 0.9720 - val_accuracy: 0.5700
Epoch 2/8
25/25 [==============================] - 53s 2s/step - loss: 0.8126 - accuracy: 0.6797 - val_loss: 0.6585 - val_accuracy: 0.7400
Epoch 3/8
25/25 [==============================] - 56s 2s/step - loss: 0.4280 - accuracy: 0.8190 - val_loss: 0.2616 - val_accuracy: 0.9075
Epoch 4/8
25/25 [==============================] - 52s 2s/step - loss: 0.3490 - accuracy: 0.8554 - val_loss: 0.6192 - val_accuracy: 0.7225
Epoch 5/8
25/25 [==============================] - 47s 2s/step - loss: 0.4350 - accuracy: 0.8276 - val_loss: 0.1624 - val_accuracy: 0.9450
Epoch 6/8
25/25 [==============================] - 42s 2s/step - loss: 0.3345 - accuracy: 0.8817 - val_loss: 0.1631 - val_accuracy: 0.9550
Epoch 7/8
25/25 [==============================] - 44s 2s/step - loss: 0.1615 - accuracy: 0.9540 - val_loss: 0.2063 - val_accuracy: 0.9050
Epoch 8/8
25/25 [==========

In [155]:
model.save("ELEHORSHECOW.h5")

In [156]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size =(224,224),seed = 42)

Found 1400 images belonging to 4 classes.


In [157]:
scores = model.evaluate(test_generator);

44/44 [==============================] - 30s 682ms/step - loss: 0.5498 - accuracy: 0.8293


In [158]:
print("Accuracy: " + str(100*scores[1]))

Accuracy: 82.9285740852356
